By [Yulandy Chiu](https://www.youtube.com/@YulandySpace)

Aided with Gemini/Claude/ChatGPT and modified by Yulandy Chiu

Version: 2025/02

Videos:
* [Python實作RAG：Google Search + Gemini完整教學 | Step-by-Step 做中學！](https://youtu.be/cmHsOdlieXY)

YouTube: [Yulandy Chiu的AI觀測站](https://www.youtube.com/@YulandySpace)

Facebook: [Yulandy Chiu的AI資訊站](https://www.facebook.com/yulandychiu)

 This code is licensed under the Creative Commons Attribution-NonCommercial 4.0
 International License (CC BY-NC 4.0). You are free to use, modify, and share this code for non-commercial purposes, provided you give appropriate credit. For more details, see the LICENSE file or visit: https://creativecommons.org/licenses/by-nc/4.0/
 © [2025] Yulandy Chiu


In [12]:
# 測試網路搜尋功能
import google.generativeai as genai
from google.colab import userdata

api_key=userdata.get('Gemini_API_Key')
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")
# response = model.generate_content("Explain how AI works")

response = model.generate_content("提供國際新聞頭條")
print(response.text)

很抱歉，我無法提供即時的國際新聞頭條。我的知識截止日期是2021年9月。要獲得最新的國際新聞頭條，我建議您參考以下可靠的新聞來源：

* **路透社 (Reuters):**  提供全球範圍的新聞報導。
* **美聯社 (Associated Press):**  另一家提供全球新聞的著名機構。
* **BBC News:**  英國廣播公司提供的新聞，涵蓋國際和國內新聞。
* **CNN:**  美國有線電視新聞網，提供國際和美國新聞。
* **紐約時報 (The New York Times):**  美國主要報紙，提供深入的新聞分析。


請前往這些網站或其他您信任的新聞來源，以獲得最新的國際新聞頭條。  請務必從多個來源獲取資訊，以獲得更全面的瞭解。



In [13]:
# Step 1: 擷取需要RAG的 query

user_query="使用繁體中文回答。根據國際新聞頭條，產生長篇故事"


import google.generativeai as genai

def generate_gemini_response(llm_input, api_key, model_name):
    """
    使用 Google Gemini API 生成回應。

    Args:
        llm_input: 用戶輸入（包含問題和檢索到的資料）。
        api_key: Google Gemini API 金鑰。
        model_name: Gemini 模型名稱，預設是 "gemini-1.5-flash"。
    Returns:
        Gemini 生成的回應。
    """
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(model_name=model_name)
    try:
        response = model.generate_content(llm_input)
        return response.text
    except Exception as e:
        print(f"Error during Gemini response generation: {e}")
        return None

# 參數修改
from google.colab import userdata
api_key=userdata.get('Gemini_API_Key')

llm_input = f"根據要求，分析需要上網查詢的資訊(內部知識不足夠)。生成需要上網查詢資訊的的關鍵字(不需要相關說明，直接回傳關鍵字，例如: 關鍵字1 關鍵字2 關鍵字3): {user_query} " # 控制输入长度在4000字符以内
model_name="gemini-1.5-flash"
Google_Search_Keywords = generate_gemini_response(llm_input, api_key, model_name)
print(Google_Search_Keywords)

國際新聞頭條  長篇故事創作  素材搜集



In [14]:
# Step 2: 檢索 Retrievel (Google Search)
# !pip install google-search-results

from googleapiclient.discovery import build

def search_google_custom(query, api_key, cse_id, num_results):
    """
    使用 Google Custom Search API 進行搜尋。

    Args:
        query: 用戶的搜尋查詢。
        api_key: Google API 金鑰。
        cse_id: Google Custom Search Engine 的 ID。
        num_results: 返回的搜尋結果數量。

    Returns:
        搜尋結果的列表，每個結果包含 title、link 和 snippet。
    """
    service = build("customsearch", "v1", developerKey=api_key)
    try:
      response = (
          service.cse()
          .list(q=query, cx=cse_id, num=num_results)
          .execute()
      )
      results = response.get("items", []) # 使用 get 方法避免当 items 不存在时报错

      formatted_results = []
      for item in results:
          formatted_results.append({
              "title": item.get("title", ""),
              "link": item.get("link", ""),
              "snippet": item.get("snippet", "")
          })
      return formatted_results
    except Exception as e:
        print(f"Error during Google Custom Search: {e}")
        return []


# 參數修改
from google.colab import userdata

api_key=userdata.get('Google_Search_API_Key')
cse_id =userdata.get('Google_CSE_ID')



search_results = search_google_custom(query, api_key, cse_id, num_results=5)

if search_results:
    for result in search_results:
        print(f"Title: {result['title']}")
        print(f"Link: {result['link']}")
        print(f"Snippet: {result['snippet']}")
        print("-" * 30)
else:
    print("No search results found.")




 # 建立 LLM 輸入字串
Retrieval_content = "這是網路上搜尋到的資料可供參考:\n"
for i, result in enumerate(search_results):
  Retrieval_content += f"標題：{result['title']}\n"
  Retrieval_content += f"摘要：{result['snippet']}\n"
print(Retrieval_content)


Title: 台湾相关报道- 美国之音中文网
Link: https://www.voachinese.com/z/1769
Snippet: 台湾 · 港澳; 国际. 最新国际新闻 印太 中东 欧洲 非洲 美洲 全球议题. 分类新闻. 经济·金融·贸易 人权·法律·宗教 科教·文娱·体健 军事 劳工 环境 · 美中关系; 热点专题.
------------------------------
Title: 台湾新闻｜台湾最新新闻｜星岛环球网
Link: https://m.stnn.cc/taiwan/
Snippet: 星岛台湾新闻频道，24小时实时更新台湾时政、台湾社会、台湾经济最新新闻资讯，聚焦台湾时政风云，关注台湾社会民生百态！
------------------------------
Title: 台灣相關報導- Yahoo奇摩新聞
Link: https://tw.news.yahoo.com/tag/%E5%8F%B0%E7%81%A3
Snippet: 最新最豐富的台灣相關新聞就在Yahoo奇摩新聞，讓你快速掌握世界大事、財經動態、體育賽事結果、影劇圈內幕、社會萬象、台灣在地訊息。
------------------------------
Title: 即時| 聯合新聞網
Link: https://udn.com/news/breaknews/1
Snippet: udn.com 提供即時新聞以及豐富的政治、社會、地方、兩岸、國際、財經、數位、運動、NBA、娛樂、生活、健康、旅遊新聞，以最即時、多元的內容，滿足行動世代的需求。
------------------------------
Title: 台湾- BBC News 中文
Link: https://www.bbc.com/zhongwen/topics/cd6qem06z92t/simp
Snippet: 台湾 · 「卡爾·文森」號航空母艦上的美國戰鬥機. 美国超级航母还有价值吗？ · 王大陆2021年参加中国长春电影节。
------------------------------
這是網路上搜尋到的資料可供參考:
標題：台湾相关报道- 美国之音中文网
摘要：台湾 · 港澳; 国际. 最新国际新闻 印太 中东 欧洲 非洲 美洲 全球议题. 分类新闻. 经济·金融·贸易 人

In [15]:
# Step 3 增加生成 augmented generation

# augmented
llm_input = f"Prompt:\n"+ f"{user_query}\n" + f"{Retrieval_content[:4000]}" # 控制输入长度在4000字符以内
print(llm_input)

# generaiton
gemini_output = generate_gemini_response(llm_input, api_key, model_name)

print(f"Gemini輸出:\n" +gemini_output)

Prompt:
使用繁體中文回答。根據國際新聞頭條，產生長篇故事
這是網路上搜尋到的資料可供參考:
標題：台湾相关报道- 美国之音中文网
摘要：台湾 · 港澳; 国际. 最新国际新闻 印太 中东 欧洲 非洲 美洲 全球议题. 分类新闻. 经济·金融·贸易 人权·法律·宗教 科教·文娱·体健 军事 劳工 环境 · 美中关系; 热点专题.
標題：台湾新闻｜台湾最新新闻｜星岛环球网
摘要：星岛台湾新闻频道，24小时实时更新台湾时政、台湾社会、台湾经济最新新闻资讯，聚焦台湾时政风云，关注台湾社会民生百态！
標題：台灣相關報導- Yahoo奇摩新聞
摘要：最新最豐富的台灣相關新聞就在Yahoo奇摩新聞，讓你快速掌握世界大事、財經動態、體育賽事結果、影劇圈內幕、社會萬象、台灣在地訊息。
標題：即時| 聯合新聞網
摘要：udn.com 提供即時新聞以及豐富的政治、社會、地方、兩岸、國際、財經、數位、運動、NBA、娛樂、生活、健康、旅遊新聞，以最即時、多元的內容，滿足行動世代的需求。
標題：台湾- BBC News 中文
摘要：台湾 · 「卡爾·文森」號航空母艦上的美國戰鬥機. 美国超级航母还有价值吗？ · 王大陆2021年参加中国长春电影节。

Gemini輸出:
海風鹹鹹的吹拂著基隆港，老漁民阿榮倚著船舷，望著遠方灰藍色的海面，心中百感交集。近來國際新聞頻頻提及台灣，像一陣陣海浪拍打著這座島嶼，帶來不安與期待。美國之音的報導提到美中關係緊張，航母部署的消息像一記重拳，震撼著島嶼的寧靜。他聽著收音機裡星島環球網的新聞播報，台灣經濟的起伏牽動著他小小的漁船，漁獲量減少，油價飆漲，日子一天比一天難過。

阿榮的孫女，小美，是個熱衷於網路新聞的年輕人。她每天瀏覽Yahoo奇摩新聞和聯合新聞網，追蹤著國際局勢的發展。她讀到BBC報導關於美軍航母的分析文章，也看到許多關於台灣社會議題的討論：年輕人的低薪、高房價，以及世代之間的價值觀衝突。她憂心忡忡，為台灣的未來感到徬徨。

這天，小美在聯合新聞網上看到一篇關於台灣半導體產業的報導，提到台灣在全球科技產業中扮演著舉足輕重的角色。這讓她稍稍振奮，但也更深切體會到台灣的脆弱與重要性。她想到爺爺阿榮，想到他日漸衰老的身體，想到漁民們樸實而堅韌的生活，也想到台灣這片土地上，人們為了生活而努力的汗水。

這天晚上，阿榮和孫女小美坐在港邊，海風輕輕吹拂

In [16]:
# 完整Rag實現
%reset -sf
user_query="使用繁體中文回答。根據2024年Donald Trump當選美國總統的新聞，產生長篇故事"


import google.generativeai as genai

def generate_gemini_response(llm_input, api_key, model_name):
    """
    使用 Google Gemini API 生成回應。

    Args:
        llm_input: 用戶輸入（包含問題和檢索到的資料）。
        api_key: Google Gemini API 金鑰。
        model_name: Gemini 模型名稱，預設是 "gemini-1.5-flash"。
    Returns:
        Gemini 生成的回應。
    """
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(model_name=model_name)
    try:
        response = model.generate_content(llm_input)
        return response.text
    except Exception as e:
        print(f"Error during Gemini response generation: {e}")
        return None

# 參數修改
from google.colab import userdata
api_key=userdata.get('Gemini_API_Key')

llm_input = f"根據要求，分析需要上網查詢的資訊(內部知識不足夠)。生成需要上網查詢資訊的的關鍵字(不需要相關說明，直接回傳關鍵字，例如: 關鍵字1 關鍵字2 關鍵字3): {user_query} " # 控制输入长度在4000字符以内
model_name="gemini-1.5-flash"
Google_Search_Keywords = generate_gemini_response(llm_input, api_key, model_name)
# print(Google_Search_Keywords)

# Step 2: 檢索 Retrievel (Google Search)
# !pip install google-search-results

from googleapiclient.discovery import build

def search_google_custom(query, api_key, cse_id, num_results):
    """
    使用 Google Custom Search API 進行搜尋。

    Args:
        query: 用戶的搜尋查詢。
        api_key: Google API 金鑰。
        cse_id: Google Custom Search Engine 的 ID。
        num_results: 返回的搜尋結果數量。

    Returns:
        搜尋結果的列表，每個結果包含 title、link 和 snippet。
    """
    service = build("customsearch", "v1", developerKey=api_key)
    try:
      response = (
          service.cse()
          .list(q=query, cx=cse_id, num=num_results)
          .execute()
      )
      results = response.get("items", []) # 使用 get 方法避免当 items 不存在时报错

      formatted_results = []
      for item in results:
          formatted_results.append({
              "title": item.get("title", ""),
              "link": item.get("link", ""),
              "snippet": item.get("snippet", "")
          })
      return formatted_results
    except Exception as e:
        print(f"Error during Google Custom Search: {e}")
        return []


# 參數修改
from google.colab import userdata

api_key=userdata.get('Google_Search_API_Key')
cse_id =userdata.get('Google_CSE_ID')
query=Google_Search_Keywords
search_results = search_google_custom(query, api_key, cse_id, num_results=5)


 # 建立 LLM 輸入字串
Retrieval_content = "這是網路上搜尋到的資料可供參考:\n"
for i, result in enumerate(search_results):
  Retrieval_content += f"標題：{result['title']}\n"
  Retrieval_content += f"摘要：{result['snippet']}\n"
# print(Retrieval_content)

# Step 3 增加生成 augmented generation

# augmented
llm_input = f"Prompt:\n"+ f"{user_query}\n" + f"{Retrieval_content[:4000]}" # 控制输入长度在4000字符以内
# print(llm_input)

# generaiton
gemini_output = generate_gemini_response(llm_input, api_key, model_name)

print(f"Gemini輸出:\n" +gemini_output)

Gemini輸出:
二〇二四年，美國大選落幕，一個讓全球屏息以待的結果出爐：唐納德·川普，再次入主白宮。

這場選舉，遠比四年前更為激烈、更為詭譎。拜登總統，儘管在競選初期展現出重振旗鼓的企圖，卻未能抵擋住時間與年齡的洪流。他那場備受期待的首次辯論，原本被視為扭轉頹勢的機會，卻因明顯的體力不支而淪為笑柄，讓支持者的心涼了半截。競選後期，拜登更因健康因素被迫退出，副總統賀錦麗臨危受命，卻難以彌補民調上的巨大差距。

川普的競選策略，則是一場充滿算計的豪賭。他無視傳統政治正確，持續煽動民粹情緒，將矛頭指向民主黨、媒體，甚至自己的共和黨內部異己。  他選擇了JD·范斯作為競選搭檔，一位擁有保守派背景的參議員，此舉被解讀為拉攏福音派及傳統保守派選民的策略。選戰期間，川普多次提及「讓美國再次偉大」的口號，並強化對華強硬立場，吸引了大量厭倦政治正確、渴望強硬領導的選民。

然而，這場選舉並非沒有陰影。俄羅斯、中國和伊朗的干預陰影始終籠罩著整個選戰過程。網路上的假訊息與抹黑攻擊如病毒般散播，讓選民難以分辨真偽。紐約時報等主流媒體持續揭露外國勢力干預的證據，但這些報導的效果似乎有限，反而被川普陣營視為「假新聞」加以攻擊。  即便如此，國際社會仍持續關注這次選舉，擔心川普的再次當選將對國際秩序產生巨大的衝擊。尤其令人擔憂的是，川普競選期間「擁抱」普丁的言論，更讓美國長期以來維繫的國際聯盟產生動搖。

選舉結果揭曉的那一刻，全球各地都為之震動。川普的支持者歡欣鼓舞，而反對者則憂心忡忡。台灣，自然也密切關注著這個結果。川普的「台灣牌」策略，以及他對中國的強硬態度，讓台灣在過去四年中經歷了前所未有的緊張與不安。然而，川普的再次當選，並不代表台灣就能高枕無憂。相反的，這也可能意味著更加複雜和不可預測的兩岸關係。川普政府將如何處理與中國的關係，如何平衡台灣的安全與美中關係的穩定，將是台灣未來四年必須密切關注的課題。

選舉過後，美國社會依然撕裂，政治分歧更加深化。 川普的再次勝選，並不代表他獲得了壓倒性的支持，而是反映出美國社會內部的深刻矛盾與日益加劇的兩極化。在接下來的四年，美國將面臨許多挑戰，包括經濟不穩定、社會分裂、以及日益嚴峻的國際局勢。而川普的領導風格，將如何應對這些挑戰，將會決定美國，以及全球未來的走向。這場大選，並非只是美國的選舉，而是世界的一場關鍵考驗。無論結果如何，它都將深刻